#### For Colab

In [2]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [3]:
!nvidia-smi

Wed Mar 17 14:42:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [6]:
!cd '/content/drive/My Drive/MAGMA: Summarization/transformers/'; pip install -q -e .
!pip install -q -U sentence-transformers
!pip install -q -U wandb
!pip install -q -U rouge-score sacrebleu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 8.4MB/s 
     |████████████████████████████████| 890kB 46.1MB/s 
     |████████████████████████████████| 71kB 5.0MB/s 
     |████████████████████████████████| 1.2MB 11.2MB/s 
     |████████████████████████████████| 2.0MB 7.6MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 133kB 37.1MB/s 
     |████████████████████████████████| 163kB 35.7MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 


In [7]:
print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

Setting up colab environment
     |████████████████████████████████| 47.5MB 55kB/s 
     |████████████████████████████████| 3.2MB 53.7MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 204kB 57.7MB/s 
     |████████████████████████████████| 1.3MB 44.2MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 133kB 62.0MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
     |████████████████████████████████| 317kB 58.2MB/s 
     |████████████████████████████████| 143kB 54.9MB/s 
     |████████████████████████████████| 296kB 58.9MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 


In [8]:
import sys

print("\n##### sys.path original #####")
for p in sys.path:
    print(f"'{p}'")
print()

import site
site.main()

print("\n##### sys.path after site.main() #####")
for p in sys.path:
    print(f"'{p}'")
print()


##### sys.path original #####
''
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'


##### sys.path after site.main() #####
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'
'/content/drive/My Drive/MAGMA: Summarization/transformers/src'



### **Config**

In [9]:
import sys
sys.path.insert(0, drive_dir)
import config

import wandb
wandb.login()

project_name = 'ft_pegasus_bull_para_embed_merged_overlaps'
%env WANDB_PROJECT=$project_name

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=ft_pegasus_para_wordembed_merge


In [10]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'
gps_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval_search.py"'

In [11]:
model_name_or_path = 'google/pegasus-large'

In [12]:
data_dir = '"'+drive_dir+'datasets/bullet_paragraph_embeddings/pegasus/st/merged_overlaps/"'

source_val_dir = data_dir[:-1] + 'val.source"'
reference_val_dir = data_dir[:-1] + 'val.target"'

source_test_dir = data_dir[:-1] + 'test.source"'
reference_test_dir = data_dir[:-1] + 'test.target"'

In [13]:
GAS = 64
LR = 5e-5

output_dir = '"'+drive_dir+'fine-tuning/'+project_name+'_gas'+\
    str(GAS)+'_lr' + str(LR) + '"'

### Evaluation Out of the Box

#### Validation

In [ ]:
eval_model_name_or_path = '"google/pegasus-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

100% 306M/306M [00:11<00:00, 26.5MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
Downloading: 100% 2.87k/2.87k [00:00<00:00, 3.98MB/s]
Downloading: 100% 2.28G/2.28G [00:58<00:00, 39.2MB/s]
Downloading: 100% 1.91M/1.91M [00:00<00:00, 45.1MB/s]
Downloading: 100% 65.0/65.0 [00:00<00:00, 111kB/s]
Downloading: 100% 88.0/88.0 [00:00<00:00, 149kB/s]
  0% 0/28 [00:00<?, ?it/s]2021-03-05 14:10:26.548710: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 28/28 [04:21<00:00,  9.34s/it]
{'rouge1_precision': 28.07, 'rouge1_recall': 33.09, 'rouge1_fmeasure': 28.59, 'rouge2_precision': 9.26, 'rouge2_recall': 10.870000000000001, 'rouge2_fmeasure': 9.39, 'rougeL_precision': 20.7, 'rougeL_recall': 24.64, 'rougeL_fmeasure': 21.14, 'rougeLsum_precision': 22.009999999999998, 'rougeLsum_recall': 25.89, 'rougeLsum_fmeasure': 22.36, 'sentence_distilroberta_cosine': 58.74093174934387, 'w2v_cosine': 99.997

#### Test

In [ ]:
eval_model_name_or_path = '"google/pegasus-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/25 [00:00<?, ?it/s]2021-03-05 14:16:26.979693: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 25/25 [03:49<00:00,  9.20s/it]
{'rouge1_precision': 24.169999999999998, 'rouge1_recall': 31.879999999999995, 'rouge1_fmeasure': 25.580000000000002, 'rouge2_precision': 8.110000000000001, 'rouge2_recall': 10.9, 'rouge2_fmeasure': 8.64, 'rougeL_precision': 18.3, 'rougeL_recall': 24.67, 'rougeL_fmeasure': 19.54, 'rougeLsum_precision': 19.57, 'rougeLsum_recall': 25.77, 'rougeLsum_fmeasure': 20.64, 'sentence_distilroberta_cosine': 55.539655685424805, 'w2v_cosine': 99.99597668647766, 'n_obs': 196, 'runtime': 229, 'seconds_per_sample': 1.1684, 'min_length': 10, 'max_length': 150}


### Fine-tuning

In [ ]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config_dir = '"'+drive_dir+'fine-tuning/config/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--freeze_embeds \
--adafactor \
--do_train \
--learning_rate $LR \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--max_steps 700 \
--logging_steps 7 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps $GAS \
--evaluation_strategy steps --eval_steps 7 --eval_beams 2 \
--predict_with_generate \
--save_steps 49 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

Streaming output truncated to the last 5000 lines.


Batches: 100%|██████████| 1/1 [00:00<00:00, 134.49it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 140.01it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 136.38it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 145.58it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 134.58it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 146.66it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 134.27it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.42it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 117.23it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 108.36it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 118.10it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 124.45it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 122.51it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.90it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.73it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 137.76it/s]


Batches: 100%|█████

### Evaluation before GPS

In [13]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-343"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_merged_overlaps_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_merged_overlaps_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_gas64_lr5e-05/checkpoint-343"

100% 306M/306M [00:12<00:00, 25.0MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/28 [00:00<?, ?it/s]2021-03-19 10:53:55.607777: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 28/28 [00:55<00:00,  1.99s/it]
{'rouge1_precision': 35.83, 'rouge1_recall': 36.61, 'rouge1_fmeasure': 33.989999999999995, 'rouge2_precision': 16.75, 'rouge2_recall': 17.49, 'rouge2_fmeasure': 15.939999999999998, 'rougeL_precision': 28.87, 'rougeL_recall': 29.970000000000002, 'rougeL_fmeasure': 27.58, 'rougeLsum_precision': 30.31, 'rougeLsum_recall': 31.25, 'rougeLsum_fmeasure': 28.92, 'sentence_distilroberta_cosine': 62.46035695075989, 'w2v_cosine': 99.99703168869019, 'n_obs': 217, 'runtime': 55, 'seconds_per_sample': 0.2535, 'min_length': 10, 'max_length': 150}


In [14]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-343"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_merged_overlaps_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_merged_overlaps_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_gas64_lr5e-05/checkpoint-343"

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/25 [00:00<?, ?it/s]2021-03-19 10:56:53.058851: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 25/25 [00:50<00:00,  2.01s/it]
{'rouge1_precision': 32.49, 'rouge1_recall': 33.43, 'rouge1_fmeasure': 30.28, 'rouge2_precision': 13.33, 'rouge2_recall': 13.79, 'rouge2_fmeasure': 12.18, 'rougeL_precision': 25.91, 'rougeL_recall': 26.950000000000003, 'rougeL_fmeasure': 24.279999999999998, 'rougeLsum_precision': 27.339999999999996, 'rougeLsum_recall': 28.189999999999998, 'rougeLsum_fmeasure': 25.480000000000004, 'sentence_distilroberta_cosine': 60.79166531562805, 'w2v_cosine': 99.99548196792603, 'n_obs': 196, 'runtime': 50, 'seconds_per_sample': 0.2551, 'min_length': 10, 'max_length': 150}


### GPS

In [ ]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-343"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_gps.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_gps.json"'

!python3 $gps_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--task summarization \
--search="min_length=5:10:15 length_penalty=0.1:1:1.9 num_beams=4:6 no_repeat_ngram_size=3:4:5" \
--dump-args \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_gas64_lr5e-05/checkpoint-343"

100% 306M/306M [00:19<00:00, 15.5MB/s]
  0% 0/28 [00:00<?, ?it/s]2021-03-07 11:10:02.042757: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 28/28 [01:22<00:00,  2.94s/it]
100% 28/28 [01:24<00:00,  3.02s/it]
100% 28/28 [01:23<00:00,  2.99s/it]
100% 28/28 [02:00<00:00,  4.31s/it]
100% 28/28 [01:59<00:00,  4.26s/it]
100% 28/28 [02:01<00:00,  4.35s/it]
100% 28/28 [01:49<00:00,  3.91s/it]
100% 28/28 [01:54<00:00,  4.10s/it]
100% 28/28 [01:55<00:00,  4.14s/it]
100% 28/28 [03:03<00:00,  6.54s/it]
100% 28/28 [03:11<00:00,  6.85s/it]
100% 28/28 [03:18<00:00,  7.10s/it]
100% 28/28 [02:12<00:00,  4.74s/it]
100% 28/28 [02:28<00:00,  5.29s/it]
100% 28/28 [02:31<00:00,  5.41s/it]
100% 28/28 [03:45<00:00,  8.05s/it]
100% 28/28 [03:54<00:00,  8.38s/it]
100% 28/28 [03:59<00:00,  8.55s/it]
100% 28/28 [01:22<00

### Evaluation

In [14]:
### Parameters found through Generative Parameter Search and visual representation
### check the file gps_pegasus_para_wordembed_merge

min_length = 10
length_penalty = 1.9
num_beams = 4
no_repeat_ngram_size = 5

In [16]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-343"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_gps_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_gps_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $min_length \
--length_penalty $length_penalty \
--num_beams $num_beams \
--no_repeat_ngram_size $no_repeat_ngram_size \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_gas64_lr5e-05/checkpoint-343"

100% 306M/306M [00:22<00:00, 13.8MB/s]
parsed the following generate kwargs: {'min_length': 10, 'length_penalty': 1.9, 'num_beams': 4, 'no_repeat_ngram_size': 5, 'max_length': 150}
  0% 0/28 [00:00<?, ?it/s]2021-03-17 14:49:19.649900: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 28/28 [02:21<00:00,  5.04s/it]
{'rouge1_precision': 26.790000000000003, 'rouge1_recall': 52.39, 'rouge1_fmeasure': 32.92, 'rouge2_precision': 12.770000000000001, 'rouge2_recall': 24.64, 'rouge2_fmeasure': 15.57, 'rougeL_precision': 20.549999999999997, 'rougeL_recall': 40.32, 'rougeL_fmeasure': 25.259999999999998, 'rougeLsum_precision': 22.97, 'rougeLsum_recall': 45.03, 'rougeLsum_fmeasure': 28.299999999999997, 'sentence_distilroberta_cosine': 66.46127104759216, 'w2v_cosine': 99.99721646308899, 'n_obs': 217, 'runtime

In [17]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-343"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_gps_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_gps_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $min_length \
--length_penalty $length_penalty \
--num_beams $num_beams \
--no_repeat_ngram_size $no_repeat_ngram_size \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_gas64_lr5e-05/checkpoint-343"

parsed the following generate kwargs: {'min_length': 10, 'length_penalty': 1.9, 'num_beams': 4, 'no_repeat_ngram_size': 5, 'max_length': 150}
  0% 0/25 [00:00<?, ?it/s]2021-03-17 14:53:46.201810: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 25/25 [02:06<00:00,  5.06s/it]
{'rouge1_precision': 21.89, 'rouge1_recall': 49.02, 'rouge1_fmeasure': 27.99, 'rouge2_precision': 8.790000000000001, 'rouge2_recall': 20.630000000000003, 'rouge2_fmeasure': 11.3, 'rougeL_precision': 16.37, 'rougeL_recall': 36.93, 'rougeL_fmeasure': 20.849999999999998, 'rougeLsum_precision': 18.77, 'rougeLsum_recall': 42.42, 'rougeLsum_fmeasure': 24.060000000000002, 'sentence_distilroberta_cosine': 64.3825888633728, 'w2v_cosine': 99.99687671661377, 'n_obs': 196, 'runtime': 126, 'seconds_per_sample': 0.6429, 'min_length': 10,